In [19]:
import pandas as pd


In [20]:
df = pd.read_csv(r"C:\D_Drive\ASM\experiment\semanticLabelingStats.csv")

In [21]:
df

,data_source,column_name,correct_type,candidate_types
0,s01.csv,Birth_URI,E67_Birth;classLink,[]
1,s01.csv,Death_URI,E69_Death;classLink,[]
2,s01.csv,Object_URI,E22_Man-Made_Object;classLink,[]
3,s01.csv,production_id,E12_Production;label,[E12_Production;label;1.0\t]
4,s01.csv,Dimensions,E54_Dimension;P3_has_note,[E54_Dimension;P3_has_note;0.8170083703369302\...
...,...,...,...,...
369,s29.csv,Medium,E55_Type;label,[E55_Type;label;0.45173166465612685\tE57_Mater...
370,s29.csv,Object Name,E55_Type;label,[E55_Type;label;0.5680582416069184\tE12_Produc...
371,s29.csv,Title,E35_Title;label,[E35_Title;label;0.4966123004859455\tE22_Man-M...
372,s29.csv,Actor_URI,E21_Person;classLink,[]


In [22]:
df = df[df.candidate_types != "[]"]

In [23]:
df

,data_source,column_name,correct_type,candidate_types
3,s01.csv,production_id,E12_Production;label,[E12_Production;label;1.0\t]
4,s01.csv,Dimensions,E54_Dimension;P3_has_note,[E54_Dimension;P3_has_note;0.8170083703369302\...
5,s01.csv,Medium,E55_Type;label,[E57_Material;label;0.4238675023076406\tE55_Ty...
6,s01.csv,Dated,E52_Time-Span;P82_at_some_time_within,[E52_Time-Span;P82_at_some_time_within;1.0\t]
7,s01.csv,Title,E35_Title;label,[E35_Title;label;0.5012069678823935\tE22_Man-M...
...,...,...,...,...
368,s29.csv,Dated,E52_Time-Span;P82_at_some_time_within,[E52_Time-Span;P82_at_some_time_within;1.0\t]
369,s29.csv,Medium,E55_Type;label,[E55_Type;label;0.45173166465612685\tE57_Mater...
370,s29.csv,Object Name,E55_Type;label,[E55_Type;label;0.5680582416069184\tE12_Produc...
371,s29.csv,Title,E35_Title;label,[E35_Title;label;0.4966123004859455\tE22_Man-M...


In [24]:
class SemanticType:
    def __init__(self, typeStr:str):
        ls = typeStr.split(";")
        self.domain = ls[0]
        self.type = ls[1]
        if len(ls) == 2:
            self.score = 1.0
        else:
            self.score = float(ls[2])

    def __repr__(self):
        return ",".join([self.domain, self.type, str(self.score)])
        

class Column:
    def __init__(self, data_source:str, column_name:str, correct_type:str, candidate_types:str):
        self.data_source = data_source
        self.column_name = column_name
        self.correct_type = SemanticType(correct_type)
        self.candidate_types = self._get_candidate_types(candidate_types)


    def _get_candidate_types(self, candidate_types:str):
        typeStr =  candidate_types.replace("[", "").replace("]", "").rstrip("\t")
        ls = typeStr.split("\t")
        return [SemanticType(ele) for ele in ls]
        


In [25]:
columns = [Column(col["data_source"], col["column_name"], col["correct_type"], col["candidate_types"]) for _,col in df.iterrows()]

In [26]:
len(columns)

288

In [27]:
len([col for col in columns if len(col.candidate_types) > 1])

209

In [28]:
len([col for col in columns if col.candidate_types[0].score > 0.5])

229

In [29]:
def f(cols):
    res = []
    for col in cols:
        candidate_types = col.candidate_types
        for i, cType in enumerate(candidate_types):
            if cType.domain == col.correct_type.domain and cType.type == col.correct_type.type and i > 0:
                res.append(col)    
    return res


len(f(columns))


30

In [36]:
for col in f([col for col in columns if len(col.candidate_types) > 1 and col.candidate_types[0].score / col.candidate_types[1].score > 2]):
    print(col.data_source, col.column_name)

s08.csv AUTHOR
s08.csv TITLE
s08.csv IMAGE__IMAGESRC
s17.csv link
s20.csv Artwork_Details
s28.csv Collector
s28.csv Nationality


In [31]:
len(f([col for col in columns if col.candidate_types[0].score > 0.6]))

3

In [15]:
for col in f([col for col in columns if col.candidate_types[0].score > 0.6]):
    print(col.data_source, col.column_name)

s08.csv AUTHOR
s08.csv IMAGE__IMAGESRC
s28.csv Nationality


In [16]:
rs = []

for col in columns:
    candidate_types = col.candidate_types
    for i, cType in enumerate(candidate_types):
        if cType.domain == col.correct_type.domain and cType.type == col.correct_type.type:
            rs.append(i)
            # print(i)


print(rs.count(0), rs.count(1), rs.count(2), rs.count(3))

258 19 7 4


19

In [17]:
print(len([col for col in columns if len(col.candidate_types) > 1 and 
    col.candidate_types[0].score / col.candidate_types[1].score > 5]))

print(len(f([col for col in columns if len(col.candidate_types) > 1 and 
    col.candidate_types[0].score / col.candidate_types[1].score > 5])))


60
0


In [18]:
print(len(f([col for col in columns if col.candidate_types[0].score > 0.2])))

30
